In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import warnings
warnings.simplefilter("ignore")

## Review

Review in slides.

## New material

### Case study: random sample of newborns

In [ ]:
births = Table.read_table('baby.csv')
births.show(5)

In [ ]:
births.select('Maternal Smoker', 
              'Birth Weight').show(5)

In [ ]:
births.select('Maternal Smoker', 
              'Birth Weight').group('Maternal Smoker')

In [ ]:
means_table = births.select('Maternal Smoker', 
              'Birth Weight').group('Maternal Smoker',np.average)
means_table

In [ ]:
births.select('Maternal Smoker', 
              'Birth Weight').hist('Birth Weight', group='Maternal Smoker')

____

In [ ]:
means_table.column(1)

In [ ]:
observed_difference = means_table.column(1).item(1)-means_table.column(1).item(0)
observed_difference

### How should we simulate under the null?
- By shuffling the group variable!

In [ ]:
births.select('Maternal Smoker').show(5)

In [ ]:
shuffled_group = births.select('Maternal Smoker').sample(with_replacement=False)
shuffled_group.show(5)

In [ ]:
simulated_births = Table().with_columns('Shuffled Maternal Smoker', shuffled_group.column(0),
                                        'Birth Weight', births.column('Birth Weight'))
simulated_births.show(5)

In [ ]:
simulated_births.group('Shuffled Maternal Smoker',np.average)

In [ ]:
means_array = simulated_births.group('Shuffled Maternal Smoker',np.average).column(1)

In [ ]:
means_array.item(0)-means_array.item(-1)

In [ ]:
def simulated_difference(table, group_label, numeric_label, function):
    """
    Takes: 
       - name of table
       - column label of numerical variable
       - column label of categorical variable
       - statistic to be calculated (as a function)
       
    Returns: Difference in statistics of the two groups
    """
    shuffled_group = table.select(group_label).sample(with_replacement=False)
    simulated_table = Table().with_columns(group_label, shuffled_group.column(0),
                                        numeric_label, table.column(numeric_label))
    
    stats_array = simulated_table.group(group_label, function).column(1)
    
    
    return stats_array.item(1)-stats_array.item(0)

In [ ]:
simulated_difference(births, 'Maternal Smoker', 'Birth Weight', np.average)

**Discussion Task** [2 min]: 
- Simulate 2500 test statistics under the null hypothesis and save them into an array called `simulated_differences.`

In [ ]:
Table().with_column('Difference Between Group Means', 
                    simulated_differences).hist()
plots.scatter(observed_difference, -0.001, color='red', s=30);
print('Our observed difference of ' + str(observed_difference) + ' is far away from what we would expect under the null!')